In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from tensorflow.keras import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from keras.utils import plot_model

Using TensorFlow backend.


In [3]:
lines=pd.read_csv("smatbot.csv",sep="^",names=["text","rpl"])
lines=lines[1:]

In [3]:
#lower casing all characters
lines.text=lines.text.apply(lambda x:str(x))
lines.rpl=lines.rpl.apply(lambda x:str(x))
lines.text=lines.text.apply(lambda x:x.lower())
lines.rpl=lines.rpl.apply(lambda x:x.lower())

def decontract(sentence):
    sentence = re.sub('\s+', ' ',sentence).strip()
    #sentence = re.sub(r"(\?)+", " ?",sentence)
    sentence = re.sub(r"won\'t", "will not", sentence)
    sentence = re.sub(r"can\'t", "cannot", sentence)
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    #sentence = re.sub(r'(\.\.+)',"",sentence)
    return sentence

#converting can't to cannot etc
lines.text=lines.text.apply(lambda x:decontract(x))
lines.rpl=lines.rpl.apply(lambda x:decontract(x))

#removing double quotations
lines.text=lines.text.apply(lambda x:re.sub("'","",x))
lines.rpl=lines.rpl.apply(lambda x:re.sub("'","",x))
lines.text=lines.text.apply(lambda x:re.sub('"',"",x))
lines.rpl=lines.rpl.apply(lambda x:re.sub('"',"",x))

#removing all special characters
exclude=set(string.punctuation)
lines.text=lines.text.apply(lambda x:"".join(ch for ch in x if ch not in exclude))
lines.rpl=lines.rpl.apply(lambda x:"".join(ch for ch in x if ch not in exclude))

#removing numbers
#lines.text=lines.text.apply(lambda x:re.sub("[0123456789]","",x))
#lines.rpl = lines.rpl.apply(lambda x: re.sub("[0123456789]", "", x))

#adding start and end tokens
lines.rpl=lines.rpl.apply(lambda x:"<SOS> "+x+" <EOS>")
lines.sample(7)
lines

,text,rpl
1,tshirt,<SOS> okay how many dress <EOS>
2,add tshirt,<SOS> okay how many dress <EOS>
3,i need tshirt,<SOS> okay how many dress <EOS>
4,discount available kya,<SOS> there are only limited products which ar...
5,any discount,<SOS> there are only limited products which ar...
6,offer,<SOS> there are only limited products which ar...
7,is there any discount,<SOS> there are only limited products which ar...
8,how to apply for offers,<SOS> there are only limited products which ar...
9,where i can find the coupon code,<SOS> there are only limited products which ar...
10,why is everyone commenting their b dates are w...,<SOS> there are only limited products which ar...


In [4]:
text_w=set()
ileng=0
for line in lines.text:
    lenofs=len(line.split())
    if lenofs>ileng:
        ileng=lenofs
    for word in line.split():
        if word not in text_w and len(word)>0:
            text_w.add(word)
rpl_w=set()
oleng=0
for line in lines.rpl:
    lenofs=len(line.split())
    if lenofs>oleng:
        oleng=lenofs
    for word in line.split():
        if word not in rpl_w:
            rpl_w.add(word)
print("total number of words in texts {}".format(len(text_w)))
print("total number of words in replies {}".format(len(rpl_w)))
print("Maximum input sequence length {}".format(ileng))
print("Maximum output sequence length {}".format(oleng))

total number of words in texts 1058
total number of words in replies 211
Maximum input sequence length 41
Maximum output sequence length 59


In [5]:
input_words=sorted(list(text_w))
target_words=sorted(list(rpl_w))
num_encoder_tokens=len(text_w)
num_decoder_tokens=len(rpl_w)
num_decoder_tokens+=1
num_encoder_tokens+=1
print((num_encoder_tokens,num_decoder_tokens))
# mapping words with indexes
#input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
#target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

(1059, 212)


In [6]:
#pre-trained word embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
inputs=lines.text.tolist()
tokenizer=Tokenizer()
tokenizer.fit_on_texts(inputs)
input_sequences=tokenizer.texts_to_sequences(inputs)
input_word_index=tokenizer.word_index

targets=lines.rpl.tolist()
tokenizer=Tokenizer()
tokenizer.fit_on_texts(targets)
target_sequences=tokenizer.texts_to_sequences(targets)
target_word_index=tokenizer.word_index

lines = shuffle(lines)
X,y=lines.text,lines.rpl
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
print((X_train.shape, X_test.shape))
X_train.to_pickle('weights_lstm/X_train.pkl')
X_test.to_pickle('weights_lstm/X_test.pkl')
target_word_index["<SOS>"]=1
target_word_index["<EOS>"]=2
del target_word_index["sos"]
del target_word_index["eos"]

((1424,), (357,))


In [7]:
glove_dir = '/root/Downloads/glove.6B'
import os
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
embedding_size=300
input_embedding_matrix=np.zeros((len(input_word_index)+1,embedding_size))
for word,i in input_word_index.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is None:
        input_embedding_matrix[i]=np.zeros((embedding_size,))
    else:
        input_embedding_matrix[i]=embedding_vector
target_embedding_matrix=np.zeros((len(target_word_index)+1,embedding_size))
for word,i in target_word_index.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is None:
        target_embedding_matrix[i]=np.zeros((embedding_size,))
    else:
        target_embedding_matrix[i]=embedding_vector
target_embedding_matrix.shape

(212, 300)

In [15]:
cnt=0
def generate_batch(X=X_train,y=y_train,batch_size=128):
    global cnt
    while True:
        for j in range(0,len(X),batch_size):
            #defining encoder and target inputs and outputs matrices
            # {ileng}--max length of input
            # {oleng}--max length of output
            encoder_input_data=np.zeros((batch_size,ileng),dtype="float32")
            decoder_input_data=np.zeros((batch_size,oleng),dtype="float32")
            decoder_target_data=np.zeros((batch_size,oleng,num_decoder_tokens),dtype="float32")
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size],y[j:j+batch_size])):
                #print(input_text,target_text)
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_word_index[word]
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word_index[word]
                    # decoder_target_data is ahead of decoder_input_data by one timestep
                    if t > 0:
                        # decoder_target_data will be ahead by one timestep
                        # and will not include the start character.
                        decoder_target_data[i, t - 1, target_word_index[word]] = 1
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [16]:
#encoder model layers
batch_size=32
encoder_inputs=Input(shape=(None,))
encoder_emb=Embedding(len(input_word_index)+1,embedding_size,input_length=batch_size)(encoder_inputs)
encoder_lstm=LSTM(embedding_size,return_state=True)
encoder_outputs,state_h,state_c=encoder_lstm(encoder_emb)
encoder_states=[state_h,state_c]
#we discard ouputs of encoders as we only need he final state
#to feed into decoder network

#decoder model layer
decoder_inputs=Input(shape=(None,))
decoder_emb_layer=Embedding(len(target_word_index)+1,embedding_size,input_length=batch_size)
decoder_emb=decoder_emb_layer(decoder_inputs)
decoder_lstm=LSTM(embedding_size, return_sequences=True, return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_emb,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation="softmax")
decoder_outputs=decoder_dense(decoder_outputs)

model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 300)    317700      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 300)    63600       input_6[0][0]                    
____________________________________________________________________________________________

In [17]:
model.layers[2].set_weights([input_embedding_matrix])
model.layers[2].trainable=False
model.layers[3].set_weights([target_embedding_matrix])
model.layers[3].trainable=False

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
train_samples=len(X_train)
val_samples=len(X_test)
print(val_samples)
epochs=100
history=model.fit_generator(generate_batch(X_train,y_train,batch_size=batch_size),
                   steps_per_epoch=train_samples//batch_size,
                   epochs=epochs,
                   validation_data=generate_batch(X_test,y_test,batch_size=batch_size),
                    validation_steps=val_samples//batch_size)

357
Epoch 1/100
44/44 [==============================] - 18s 417ms/step - loss: 0.8012 - acc: 0.1467 - val_loss: 0.4068 - val_acc: 0.2144
Epoch 2/100
44/44 [==============================] - 16s 362ms/step - loss: 0.2908 - acc: 0.2441 - val_loss: 0.1653 - val_acc: 0.2448
Epoch 3/100
44/44 [==============================] - 16s 361ms/step - loss: 0.1307 - acc: 0.2743 - val_loss: 0.0829 - val_acc: 0.2515
Epoch 4/100
44/44 [==============================] - 16s 366ms/step - loss: 0.0750 - acc: 0.2796 - val_loss: 0.0615 - val_acc: 0.2546
Epoch 5/100
44/44 [==============================] - 16s 367ms/step - loss: 0.0563 - acc: 0.2807 - val_loss: 0.0504 - val_acc: 0.2563
Epoch 6/100
44/44 [==============================] - 16s 367ms/step - loss: 0.0496 - acc: 0.2817 - val_loss: 0.0465 - val_acc: 0.2561
Epoch 7/100
44/44 [==============================] - 16s 367ms/step - loss: 0.0470 - acc: 0.2802 - val_loss: 0.0491 - val_acc: 0.2753
Epoch 8/100
44/44 [==============================] - 16s 3

Epoch 62/100
44/44 [==============================] - 19s 440ms/step - loss: 0.0015 - acc: 0.2984 - val_loss: 0.0047 - val_acc: 0.2711
Epoch 63/100
44/44 [==============================] - 22s 498ms/step - loss: 0.0020 - acc: 0.2981 - val_loss: 0.0054 - val_acc: 0.2699
Epoch 64/100
44/44 [==============================] - 16s 369ms/step - loss: 0.0013 - acc: 0.2969 - val_loss: 0.0049 - val_acc: 0.2696
Epoch 65/100
44/44 [==============================] - 16s 356ms/step - loss: 0.0015 - acc: 0.2971 - val_loss: 0.0063 - val_acc: 0.2698
Epoch 66/100
44/44 [==============================] - 16s 354ms/step - loss: 0.0021 - acc: 0.2973 - val_loss: 0.0055 - val_acc: 0.2709
Epoch 67/100
44/44 [==============================] - 16s 355ms/step - loss: 0.0012 - acc: 0.2971 - val_loss: 0.0049 - val_acc: 0.2909
Epoch 68/100
44/44 [==============================] - 16s 356ms/step - loss: 7.8868e-04 - acc: 0.2977 - val_loss: 0.0044 - val_acc: 0.2708
Epoch 69/100
44/44 [==============================]

In [19]:
#inference model to predict outputs
#get the state given the input sequence 
encoder_model=Model(encoder_inputs,encoder_states)
#decoder setup
decoder_state_input_h=Input(shape=(embedding_size,))
decoder_state_input_c=Input(shape=(embedding_size,))
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]
decoder_emb2=decoder_emb_layer(decoder_inputs)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(decoder_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [20]:
model.save_weights('lstmchat.h5')

In [21]:
reverse_input_char_index = dict((i, word) for word, i in input_word_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_word_index.items())
def decode(input_seq):
    # Encode the input as state vectors
    states_value=encoder_model.predict(input_seq)
    # Generate empty target squence of length 1
    target_seq=np.zeros((1,1))
    #Give the first word of target sequence as start token
    target_seq[0,0]=target_word_index["<SOS>"]
    
    #sampling loop for a batch of sequences
    stop=False
    decoded_sent=""
    while not stop:
        # we are predicting the final state vector of encoder i.e., {states_value}
        # and initially target_seq will be start token <sos>
        output_tokens,h,c=decoder_model.predict([target_seq]+states_value)
        sampled_token_index=np.argmax(output_tokens[0,:,:])
        #print(sampled_token_index)
        sampled_word=reverse_target_char_index[sampled_token_index]
        decoded_sent+=" "+sampled_word
        
        # stop loop if <EOS> token is encountered or max length
        if (sampled_word=="<EOS>" or len(decoded_sent)>1000):
            stop=True
        #update the target sequence to the previous output
        target_seq=np.zeros((1,1))
        target_seq[0,0]=sampled_token_index
        states_value=[h,c]
    return decoded_sent      

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
print(input_seq.shape)
decoded_sentence = decode(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-5])
print('Predicted Marathi Translation:', decoded_sentence)

In [43]:
inp="where is store"
input_seq=np.zeros((1,ileng),dtype="float32")
print(input_seq.shape)
seq=[]
for i,word in enumerate(inp.split()):
    input_seq[0,i]=input_word_index[word]
decoded_sentence = decode(input_seq)
print('Predicted Marathi Translation:', decoded_sentence[:-5])

(1, 41)
Predicted Marathi Translation:  we do not have any physical store you can place your order here online url httpsmydreamstorein 
